# Prepare defects for iprhub

This Notebook is for preparing all of the material defect calculations for the data on iprhub. It is being done here as I'm calling each defect calculation's prepare function directly and passing in only one calculation-refine-structure record per potential, crystal prototype, composition.

Load Python packages

In [1]:
#Standard library imports
from __future__ import print_function, division
import os
import sys
import glob
import uuid
import shutil

#http://www.numpy.org/
import numpy as np

#http://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

#https://github.com/usnistgov/atomman
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

#https://github.com/usnistgov/iprPy
import iprPy

## Get records from database

Load local copy of iprhub calculations

In [2]:
dbase = iprPy.Database('local', 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\library')

Build dataframe of all calculation-system-relax records

In [3]:
record_df = []
for record in dbase.iget_records(style='calculation-system-relax'):
    record_df.append(record.todict(full=False))
record_df = pd.DataFrame(record_df)
print(len(record_df), 'records loaded')

Exception RuntimeError: 'generator ignored GeneratorExit' in <generator object iaslist at 0x000000000D85D438> ignored
Exception RuntimeError: 'generator ignored GeneratorExit' in <generator object iaslist at 0x000000000D85D2D0> ignored


ValueError: Unable to parse as JSON or XML

## Select which records to use

Define data selection limiting conditions

In [21]:
# Limit to calc_refine_structure 
condition1 = record_df.calc_script == 'calc_refine_structure'

# Limit to only one strainrange
condition2 = np.isclose(record_df.strainrange, 1e-7, rtol=0, atol=1e-9)

# Limit to only the prototypes that have defect models (currently)
condition3 = record_df.prototype.isin(['A1--Cu--fcc', 'A2--W--bcc'])

# Limit to only calculations that successfully finished
condition4 = record_df.status == 'finished'

# Limit to only zero pressure runs
condition5 = np.isclose(record_df.pressure_xx, 0.0, rtol=0, atol=1e-9)
condition6 = np.isclose(record_df.pressure_yy, 0.0, rtol=0, atol=1e-9)
condition7 = np.isclose(record_df.pressure_zz, 0.0, rtol=0, atol=1e-9)

# Limit to only runs with 3x3x3 supercells
condition8 = np.empty(len(record_df), dtype=bool)
for i in xrange(len(record_df)):
    condition8[i] = np.allclose(record_df.iloc[i].sizemults, np.array([[0,3],[0,3],[0,3]]))

Apply limiters 

In [22]:
starting_record_df = record_df[(condition1 & condition2 & condition3 & condition4 & 
                                condition5 & condition6 & condition7 & condition8)]
print(len(starting_record_df), 'calculations after limiting conditions')

(426, 'calculations after limiting conditions')


Make list of selected records

In [24]:
records = []
for record_key in starting_record_df.calc_key.tolist():
    records.append(dbase.get_record(name=record_key))

## Prepare defect calculations

### Prepare surface_energy

Define prepare function variables

In [25]:
#Specify calculation style
calculation_style = 'surface_energy'

#Specify run directory for the prepared calculations
run_directory = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub-local\\1'

#Define any additional prepare parameters
input_params = {}
input_params['lammps_command'] = 'lmp_serial'
input_params['sizemults'] =      '5 5 5'

Call prepare function

In [27]:
calc = iprPy.Calculation(calculation_style)
calc.prepare(dbase, run_directory, parent_records=records, **input_params)

### Prepare stacking_fault

Define prepare function variables

In [25]:
#Specify calculation style
calculation_style = 'stacking_fault_multi'

#Specify run directory for the prepared calculations
run_directory = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\torun\\iprhub-local\\4'

#Define any additional prepare parameters
input_params = {}
input_params['lammps_command'] =           'lmp_serial'
input_params['sizemults'] =                '20 20 20'
input_params['stackingfault_numshifts1'] = '51'
input_params['stackingfault_numshifts2'] = '51'
input_params['keepatomfiles'] =            'False'


Call prepare function

In [27]:
calc = iprPy.Calculation(calculation_style)
calc.prepare(dbase, run_directory, parent_records=records, **input_params)